HTTP/2的目的是通过支持完整的请求与响应服用来减少延迟，通过有效压缩`HTTP`标头字段将协议开销降至最低，同时增加对请求优先级和服务器推送的支持。

为达成这些目标，HTTP/2还给我们带来了大量其他协议层面的辅助实现，例如新的流控制、错误处理和升级机制。

HTTP/2没有改动HTTP的应用语义。HTTP方法、状态代码、URI和标头字段等核心一如往常。不过，`HTTP/2`修改了数据格式化（分帧）以及在客户端与服务器间传输的方式。这两点统帅全局，通过新的分帧层向我们的应用隐藏了所有复杂性。因此，**所有现有的应用都可以不必修改而在新协议下运行**。

## HTTP/1.x

1. 客户端需要使用多个连接才能实现并发和缩短延迟
2. 不会压缩请求和响应标头，从而导致不必要的网络流量
3. 不支持有效的资源优先级，致使底层TCP连接的利用率低下

HTTP/2通过支持标头字段压缩和在**同一连接上进行多个并发交换**，让应用更有效地利用网络资源，减少感知的延迟时间。具体来说，它可以在对同一个连接上的请求和响应消息进行交错发送并为`HTTP`标头字段使用有效编码。HTTP/2还允许为请求设置优先级，让更重要的请求更快速地完成，从而进一步提升性能。出台地协议对网络更加友好，因为与`HTTP/1.x`相比，可以使用更少地TCP连接。

![](https://developers.google.com/web/fundamentals/performance/http2/images/binary_framing_layer01.svg?hl=zh-cn)

HTTP/2所有性能增强地核心在于新的二进制分帧层，它定义了如何封装HTTP消息并在客户端与服务器之间传输

这里所谓的“层”，指的是位于套接字接口与应用可见的高级HTTP API之间一个晋国优化的新编码机制，不同的是传输期间对它们的编码方式变了。HTTP/1.x协议以换行符作为纯文本的分隔符好，而HTTP/2将所有传输的信息分割为更小的消息和帧，并采用二进制格式对它们编码。

这样以来，客户端和服务器为了相互理解，都必须使用新的二进制编码机制：`HTTP/1.x`客户端无法理解只支持`HTTP/2`的服务器，反之亦然。不过不要紧，现有的应用不必担心这些变化，因为客户端和服务器会替我们完成必要的分帧工作。

新的二进制分帧机制改变了客户端与服务器之间交换数据的方式。为了说明这个过程，我们需要了解`HTTP/2`的三个概念：

数据流： 已建立的连接内的双向字节流，可以承载一条或多条消息。
消息： 与逻辑请求或响应消息对应的完整的一系列帧
帧： `HTTP/2`通信的最小单位，每个帧都包含帧头，至少也会标识处当前帧所属的数据流。

这些概念的关系总结如下：

1. **所有通信都在一个TCP连接上完成**，此链接可以承载任意数量的双向数据流
2. 每个数据流都有一个唯一的标识符和可选的优先级信息，用于承载双向消息
3. 每条消息都是一条逻辑`HTTP`消息（例如请求或响应），包含一个或多个帧
4. 帧是最小的通信单位，承载着特定类型的数据，例如HTTP标头、消息负载，等等。来自不同数据流的帧可以交错发送，然后再更具每个帧头的数据流标识符重新组装

![](https://developers.google.com/web/fundamentals/performance/http2/images/streams_messages_frames01.svg?hl=zh-cn)

简而言之，`HTTP/2`将HTTP协议通信分解为二进制编码帧的交换，这些帧对应着特定数据流中的消息，所有这些都在一个TCP连接内服用。这是`HTTP/2`协议所有其他功能和性能优化的基础